In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle
import gzip
from collections import Counter
%matplotlib inline

In [ ]:
## 데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/BOAZ/mini/청와대 청원/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/BOAZ/mini/청와대 청원/test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/drive/MyDrive/BOAZ/mini/청와대 청원/sample_submission.csv', encoding = 'utf-8')

In [ ]:
## konlpy, mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2023-02-25 10:03:04--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-02-25 10:03:04--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9870c0222f56c4efc8be5c6b4a.dl.dropboxusercontent.com/cd/0/get/B3LKBoONt10rAPP4_nEETS1o593i1iTc0t_V7tqXAAyT0qZVBI34_Y5QeN8C_-64DFbMoUZmHdCxwztxpFmPdlX-Clq1xm4YDUbQuqpmJDG-PmZq-Pp74ma_2e_5McKHccmTPpfNRibqpRh3AmMoixQH_prOglV4FOaVLE6jodA77avIVFC-yA-kNQyVvPDTr8c/file?dl=1# [following]
--2023-02-25 10:03:04--  https://uc9870c0222f56c4efc8be5c6b4a.dl.dropboxusercontent.com/cd/0/get/B3LKBoONt10r

# 전처리

## 결측치, 불용어 제거(1)

In [ ]:
np.sum(train.isnull())

index       0
category    0
data        8
dtype: int64

In [ ]:
train.dropna(axis=0, inplace=True)

In [ ]:
np.sum(test.isnull())

index    0
data     0
dtype: int64

In [ ]:
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

# 특수문자 제거
train['data'] = train['data'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
test['data'] = test['data'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', " ", regex=True)
train['data'] = train['data'].str.replace(r'\t+', " ", regex=True)
test['data'] = test['data'].str.replace(r'\t+', " ", regex=True)
train['data'] = train['data'].str.replace(r'[\\n]+'," ", regex=True)
test['data'] = test['data'].str.replace(r'[\\n]+'," ", regex=True)

<ipython-input-9-709a20f71fbe>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-9-709a20f71fbe>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


## 가장 짧은 문장 제거

In [ ]:
## 가장 짧은 문장은 삭제(길이=1)
pop_index = list(np.where(train['data'].apply(len)==1)[0])

In [ ]:
train.drop(pop_index,inplace=True)
train.reset_index(inplace=True,drop=True)

In [ ]:
## 그 다음으로 짧은 문장 삭제(길이=3)
pop_index = list(np.where(train['data'].apply(len)==3)[0])
train.drop(pop_index,inplace=True)
train.reset_index(inplace=True,drop=True)

## 불용어 제거(2)

- 한국어 불용어 100 일부 추가 ([출처](https://bab2min.tistory.com/544))

In [ ]:
from konlpy.tag import Mecab

In [ ]:
X_train = []
stopwords = ['과', '우리', '나', '좀', '지금', '때', '말', '것', '자', '의', '두', '로', '하나', '명', '살', '건', '를', '부', '중', 
             '고', '위하', '을', '년', '하다', '때문', '저', '들', '데', '게', '개', '도', '또', '일', '경우', '있', '그러나', 
             '더', '되', '한', '그', '그리고', '주', '그런', '정도', '어떻', '시키', '적', '이런', '잘', '같', '에', '오', '이', '걍', 
             '는', '자신', '원', '가', '월', '으로', '아니', '지', '은', '그렇', '보', '않', '씨', '그것', '어떤', '전', '안', '놓', 
             '점', '후', '만', '하', '이렇', '없', '속', '다시', '등', '거', '와', '그러', '싶', '내', '저희', '시간', '제', '수', 
             '알', '분', '번','합니다','입니다','이다','습니다','입니까','습니까','세요','해요','다고','한다고']

mecab = Mecab()

for sentence in train['data'] :
    temp_X = []
    temp_X = mecab.morphs(sentence)

    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

In [ ]:
X_test = []
stopwords = ['과', '우리', '나', '좀', '지금', '때', '말', '것', '자', '의', '두', '로', '하나', '명', '살', '건', '를', '부', '중', 
             '고', '위하', '을', '년', '하다', '때문', '저', '들', '데', '게', '개', '도', '또', '일', '경우', '있', '그러나', 
             '더', '되', '한', '그', '그리고', '주', '그런', '정도', '어떻', '시키', '적', '이런', '잘', '같', '에', '오', '이', '걍', 
             '는', '자신', '원', '가', '월', '으로', '아니', '지', '은', '그렇', '보', '않', '씨', '그것', '어떤', '전', '안', '놓', 
             '점', '후', '만', '하', '이렇', '없', '속', '다시', '등', '거', '와', '그러', '싶', '내', '저희', '시간', '제', '수', 
             '알', '분', '번','합니다','입니다','이다','습니다','입니까','습니까','세요','해요','다고','한다고']

mecab = Mecab()

for sentence in test['data'] :
    temp_X = []
    temp_X = mecab.morphs(sentence)

    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

## Tokenizer
- 아래는 **baseline code** 그대로 가져온 것입니다

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
vocab_size = 30000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# 각 단어별 빈도수 확인
print(tokenizer.word_counts.items())

odict_items([('신혼', 123), ('부부', 1238), ('위한', 5473), ('주택', 468), ('정책', 5947), ('보다', 7645), ('보육', 3841), ('시설', 1921), ('늘려', 731), ('국민', 20771), ('세금', 2850), ('일부', 2599), ('펴', 220), ('마시', 1131), ('보편', 235), ('모든', 7740), ('수긍', 77), ('할', 35617), ('복지', 1869), ('시', 25148), ('길', 6790), ('바랍니다', 6269), ('지만', 17719), ('당첨', 90), ('사람', 23727), ('로또', 214), ('반대', 5201), ('에게', 26158), ('퍼', 214), ('기', 37953), ('식', 2846), ('면', 32928), ('죠', 7006), ('아이', 27014), ('전하', 565), ('맡길', 298), ('전국', 1841), ('설치', 1196), ('해', 45771), ('대', 7278), ('기업', 2106), ('솔선수범', 24), ('해서', 11641), ('사업장', 133), ('의무', 4818), ('도록', 10987), ('구', 1036), ('요집', 10), ('애', 3241), ('어', 40300), ('경력', 776), ('단절', 306), ('괴롭', 110), ('집', 4651), ('개인', 4826), ('능력', 2070), ('키워', 282), ('사', 5104), ('맞', 4970), ('키울', 465), ('육아', 2141), ('전담', 271), ('힘', 2491), ('기울이', 98), ('부모', 7984), ('키우', 2579), ('이제', 4121), ('국가', 13433), ('책임지', 465), ('시대', 2459), ('봅니다', 3807), ('잖아도', 6), ('부동

In [ ]:
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
y_train = to_categorical(train['category'])
y_test = to_categorical(test['index'])

##LSTM

## 모델 생성 ( epoch = 2 )
embedding_dim = 120
hidden_units = 120

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras import layers

In [ ]:
embedding_dim = 120
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=2)

Epoch 1/2
313/313 [==============================] - 66s 187ms/step - loss: 0.6577 - acc: 0.7381
Epoch 2/2
313/313 [==============================] - 39s 123ms/step - loss: 0.3893 - acc: 0.8680


## 모델 예측

In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM120_2.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 10ms/step


epoch = 2일 때, LSTM의 정확도 : 85.02%

In [ ]:
#loaded_model = load_model('best_model.h5')
#print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test,y_pred)[1]))

OSError: ignored

## 모델 생성 ( epoch = 2 )
embedding_dim = 150
hidden_units = 120

In [ ]:
embedding_dim = 150
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=2)

Epoch 1/2
313/313 [==============================] - 62s 193ms/step - loss: 0.5452 - acc: 0.7858
Epoch 2/2
313/313 [==============================] - 42s 132ms/step - loss: 0.2958 - acc: 0.8966


##모델 예측2

In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM150_2.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 8ms/step


epoch = 2일 때, LSTM의 정확도 : 86.97%

## 모델 생성 ( epoch = 2 )
embedding_dim = 200
hidden_units = 120

In [ ]:
embedding_dim = 200
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=2)

Epoch 1/2
313/313 [==============================] - 65s 196ms/step - loss: 0.5139 - acc: 0.8109
Epoch 2/2
313/313 [==============================] - 54s 173ms/step - loss: 0.2919 - acc: 0.8972


In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM200_2.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 8ms/step


epoch = 2일 때, LSTM의 정확도 : 84.14%



## 모델 생성 ( epoch = 5 )
embedding_dim = 120
hidden_units = 120

In [ ]:
embedding_dim = 120
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=5)

Epoch 1/5
313/313 [==============================] - 62s 189ms/step - loss: 0.6209 - acc: 0.7593
Epoch 2/5
313/313 [==============================] - 37s 118ms/step - loss: 0.3353 - acc: 0.8878
Epoch 3/5
313/313 [==============================] - 30s 96ms/step - loss: 0.2461 - acc: 0.9169
Epoch 4/5
313/313 [==============================] - 27s 87ms/step - loss: 0.1890 - acc: 0.9365
Epoch 5/5
313/313 [==============================] - 18s 57ms/step - loss: 0.1550 - acc: 0.9467


In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM120_5.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 10ms/step


epoch = 5일 때, LSTM의 정확도 : 85.78%

## 모델 생성 ( epoch = 5 )
embedding_dim = 150
hidden_units = 120

In [ ]:
embedding_dim = 150
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=5)

Epoch 1/5
313/313 [==============================] - 42s 126ms/step - loss: 0.5180 - acc: 0.8055
Epoch 2/5
313/313 [==============================] - 31s 98ms/step - loss: 0.3320 - acc: 0.8841
Epoch 3/5
313/313 [==============================] - 23s 75ms/step - loss: 0.2247 - acc: 0.9229
Epoch 4/5
313/313 [==============================] - 23s 74ms/step - loss: 0.1823 - acc: 0.9359
Epoch 5/5
313/313 [==============================] - 18s 57ms/step - loss: 0.1504 - acc: 0.9471


In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM150_5.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 8ms/step


epoch = 5일 때, LSTM의 정확도 : 91.21%

## 모델 생성 ( epoch = 5 )
embedding_dim = 200
hidden_units = 120

In [ ]:
embedding_dim = 200
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=5)

Epoch 1/5
313/313 [==============================] - 40s 121ms/step - loss: 0.4904 - acc: 0.8150
Epoch 2/5
313/313 [==============================] - 32s 101ms/step - loss: 0.2866 - acc: 0.9020
Epoch 3/5
313/313 [==============================] - 24s 78ms/step - loss: 0.2132 - acc: 0.9246
Epoch 4/5
313/313 [==============================] - 22s 70ms/step - loss: 0.1669 - acc: 0.9400
Epoch 5/5
313/313 [==============================] - 21s 66ms/step - loss: 0.2032 - acc: 0.9311


In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM200_5.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 10ms/step


epoch = 5일 때, LSTM의 테스트정확도 : 88.7%

## 모델 생성 ( epoch = 10 )
embedding_dim = 120
hidden_units = 120

In [ ]:
embedding_dim = 120
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
313/313 [==============================] - 40s 121ms/step - loss: 0.5153 - acc: 0.8000
Epoch 2/10
313/313 [==============================] - 29s 93ms/step - loss: 0.2960 - acc: 0.8972
Epoch 3/10
313/313 [==============================] - 24s 77ms/step - loss: 0.2568 - acc: 0.9151
Epoch 4/10
313/313 [==============================] - 19s 61ms/step - loss: 0.2402 - acc: 0.9198
Epoch 5/10
313/313 [==============================] - 18s 57ms/step - loss: 0.1705 - acc: 0.9400
Epoch 6/10
313/313 [==============================] - 17s 55ms/step - loss: 0.1414 - acc: 0.9511
Epoch 7/10
313/313 [==============================] - 15s 48ms/step - loss: 0.1274 - acc: 0.9548
Epoch 8/10
313/313 [==============================] - 15s 47ms/step - loss: 0.1710 - acc: 0.9420
Epoch 9/10
313/313 [==============================] - 13s 42ms/step - loss: 0.1530 - acc: 0.9497
Epoch 10/10
313/313 [==============================] - 13s 40ms/step - loss: 0.1051 - acc: 0.9631


In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM120_10.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 8ms/step


epoch = 10일 때, LSTM의 정확도 : 91.08%

> 들여쓴 블록



## 모델 생성 ( epoch = 10 )
embedding_dim = 150
hidden_units = 120

In [ ]:
embedding_dim = 150
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
313/313 [==============================] - 41s 125ms/step - loss: 0.5136 - acc: 0.7958
Epoch 2/10
313/313 [==============================] - 29s 92ms/step - loss: 0.2801 - acc: 0.8993
Epoch 3/10
313/313 [==============================] - 25s 78ms/step - loss: 0.2183 - acc: 0.9229
Epoch 4/10
313/313 [==============================] - 22s 71ms/step - loss: 0.1775 - acc: 0.9371
Epoch 5/10
313/313 [==============================] - 19s 59ms/step - loss: 0.1569 - acc: 0.9453
Epoch 6/10
313/313 [==============================] - 18s 57ms/step - loss: 0.1634 - acc: 0.9408
Epoch 7/10
313/313 [==============================] - 15s 49ms/step - loss: 0.1082 - acc: 0.9602
Epoch 8/10
313/313 [==============================] - 15s 48ms/step - loss: 0.1080 - acc: 0.9625
Epoch 9/10
313/313 [==============================] - 14s 46ms/step - loss: 0.2848 - acc: 0.9112
Epoch 10/10
313/313 [==============================] - 15s 48ms/step - loss: 0.2077 - acc: 0.9332


In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM150_10.csv', encoding='utf-8', index = False)

epoch = 10일 때, LSTM의 정확도 : 89.28%

## 모델 생성 ( epoch = 10 )
embedding_dim = 200
hidden_units = 120

In [ ]:
embedding_dim = 200
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
313/313 [==============================] - 48s 147ms/step - loss: 0.5146 - acc: 0.8052
Epoch 2/10
313/313 [==============================] - 37s 119ms/step - loss: 0.2818 - acc: 0.9001
Epoch 3/10
313/313 [==============================] - 28s 91ms/step - loss: 0.2262 - acc: 0.9195
Epoch 4/10
313/313 [==============================] - 26s 84ms/step - loss: 0.1816 - acc: 0.9356
Epoch 5/10
313/313 [==============================] - 22s 71ms/step - loss: 0.1463 - acc: 0.9473
Epoch 6/10
313/313 [==============================] - 21s 67ms/step - loss: 0.1255 - acc: 0.9546
Epoch 7/10
313/313 [==============================] - 20s 63ms/step - loss: 0.1247 - acc: 0.9548
Epoch 8/10
313/313 [==============================] - 18s 56ms/step - loss: 0.1352 - acc: 0.9529
Epoch 9/10
313/313 [==============================] - 17s 53ms/step - loss: 0.0928 - acc: 0.9656
Epoch 10/10
313/313 [==============================] - 15s 49ms/step - loss: 0.0897 - acc: 0.9682


In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM200_10.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 9ms/step


epoch = 10일 때, LSTM의 정확도 : 91.12%

## 모델 생성 ( epoch = 15 )
embedding_dim = 120
hidden_units = 120

In [ ]:
embedding_dim = 120
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
313/313 [==============================] - 41s 124ms/step - loss: 0.5042 - acc: 0.8047
Epoch 2/15
313/313 [==============================] - 32s 101ms/step - loss: 0.3123 - acc: 0.8938
Epoch 3/15
313/313 [==============================] - 24s 77ms/step - loss: 0.2162 - acc: 0.9239
Epoch 4/15
313/313 [==============================] - 21s 67ms/step - loss: 0.1762 - acc: 0.9386
Epoch 5/15
313/313 [==============================] - 19s 61ms/step - loss: 0.1556 - acc: 0.9446
Epoch 6/15
313/313 [==============================] - 18s 57ms/step - loss: 0.1895 - acc: 0.9400
Epoch 7/15
313/313 [==============================] - 17s 53ms/step - loss: 0.1374 - acc: 0.9519
Epoch 8/15
313/313 [==============================] - 14s 44ms/step - loss: 0.1155 - acc: 0.9598
Epoch 9/15
313/313 [==============================] - 14s 44ms/step - loss: 0.1175 - acc: 0.9605
Epoch 10/15
313/313 [==============================] - 15s 47ms/step - loss: 0.0911 - acc: 0.9681
Epoch 11/15
313/313 [======

In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM120_15.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 8ms/step


epoch = 15일 때, LSTM의 정확도 : 92.64%

## 모델 생성 ( epoch = 15 )
embedding_dim = 150
hidden_units = 120

In [ ]:
embedding_dim = 150
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
313/313 [==============================] - 55s 166ms/step - loss: 0.5464 - acc: 0.7925
Epoch 2/15
313/313 [==============================] - 37s 118ms/step - loss: 0.3080 - acc: 0.8934
Epoch 3/15
313/313 [==============================] - 32s 103ms/step - loss: 0.2215 - acc: 0.9222
Epoch 4/15
313/313 [==============================] - 25s 79ms/step - loss: 0.1718 - acc: 0.9399
Epoch 5/15
313/313 [==============================] - 21s 66ms/step - loss: 0.1568 - acc: 0.9453
Epoch 6/15
313/313 [==============================] - 19s 62ms/step - loss: 0.1585 - acc: 0.9444
Epoch 7/15
313/313 [==============================] - 20s 64ms/step - loss: 0.1677 - acc: 0.9424
Epoch 8/15
313/313 [==============================] - 18s 58ms/step - loss: 0.1165 - acc: 0.9596
Epoch 9/15
313/313 [==============================] - 16s 51ms/step - loss: 0.0886 - acc: 0.9688
Epoch 10/15
313/313 [==============================] - 16s 52ms/step - loss: 0.1023 - acc: 0.9632
Epoch 11/15
313/313 [=====

In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM150_15.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 8ms/step


epoch = 15일 때, LSTM의 정확도 : 94.36%

## 모델 생성 ( epoch = 15 )
embedding_dim = 200
hidden_units = 120

In [ ]:
embedding_dim = 200
hidden_units = 120

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
313/313 [==============================] - 42s 127ms/step - loss: 0.5266 - acc: 0.8000
Epoch 2/15
313/313 [==============================] - 30s 96ms/step - loss: 0.2870 - acc: 0.8990
Epoch 3/15
313/313 [==============================] - 28s 88ms/step - loss: 0.2207 - acc: 0.9231
Epoch 4/15
313/313 [==============================] - 21s 67ms/step - loss: 0.1962 - acc: 0.9309
Epoch 5/15
313/313 [==============================] - 19s 61ms/step - loss: 0.1624 - acc: 0.9438
Epoch 6/15
313/313 [==============================] - 19s 62ms/step - loss: 0.1325 - acc: 0.9549
Epoch 7/15
313/313 [==============================] - 17s 56ms/step - loss: 0.2056 - acc: 0.9337
Epoch 8/15
313/313 [==============================] - 16s 52ms/step - loss: 0.2723 - acc: 0.9106
Epoch 9/15
313/313 [==============================] - 16s 53ms/step - loss: 0.2332 - acc: 0.9188
Epoch 10/15
313/313 [==============================] - 16s 52ms/step - loss: 0.1304 - acc: 0.9525
Epoch 11/15
313/313 [=======

In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('LSTM200_15.csv', encoding='utf-8', index = False)

157/157 [==============================] - 2s 7ms/step


epoch = 15일 때, LSTM의 정확도 : 90.42%